# Neural Network for Music Speech Separation

In [3]:
import tensorflow as tf

2022-08-04 14:18:53.803016: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-08-04 14:18:53.803190: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [4]:
from tensorflow import keras

In [5]:
import soundfile as sf
import librosa as lb
import numpy as np
from matplotlib import pyplot as plt

In [6]:
import os

## Reading DataSet

In [7]:
path = "/home/rajesh/MusicSpeechSeparartion/Dataset/"

folders = os.listdir(path)
folders

['Overlay', 'Mixture', 'speech', 'music']

In [8]:
audiofiles, samplerates = [], []
for folder in folders:
    files = os.listdir(path+'/'+folder)
    files = sorted(files)
    audio, sr = [], []
    for file in files:
        data, samplerate = sf.read(path+'/'+folder+'/'+file)
        audio.append(data)
        sr.append(samplerate)
    audiofiles.append(audio)
    samplerates.append(sr)

In [9]:
overlay = np.array(audiofiles[0])
mixture = np.array(audiofiles[1])
speech = np.array(audiofiles[2])
music = np.array(audiofiles[3])

dataset = [overlay, mixture, speech, music]

### Short Time Fourier Transform

In [10]:
datasetstft = []
for data in dataset:
    spect = []
    for audio in data:
        stft = np.abs(lb.stft(audio))
        spect.append(stft)
    datasetstft.append(spect)

In [11]:
overlay_stft = np.array(datasetstft[0])
mixture_stft = np.array(datasetstft[1])
speech_stft = np.array(datasetstft[2])
music_stft = np.array(datasetstft[3])

dataset_stft = [overlay_stft, mixture_stft, speech_stft, music_stft]

In [12]:
overlay_stft.shape

(60, 1025, 646)

## DNN Model

In [15]:
model = keras.Sequential([
    keras.layers.Reshape(target_shape=(1025 * 646,), input_shape=(1025, 646)),
    keras.layers.Dense(units=256, activation='relu'),
    keras.layers.Dense(units=192, activation='relu'),
    #keras.layers.Dense(units=128, activation='relu'),
    keras.layers.Dense(units=2, activation='softmax')
])

2022-08-04 14:21:04.590608: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 678041600 exceeds 10% of free system memory.
2022-08-04 14:21:04.812849: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 678041600 exceeds 10% of free system memory.
2022-08-04 14:21:04.893694: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 678041600 exceeds 10% of free system memory.


In [20]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 reshape_2 (Reshape)         (None, 662150)            0         
                                                                 
 dense_7 (Dense)             (None, 256)               169510656 
                                                                 
 dense_8 (Dense)             (None, 192)               49344     
                                                                 
 dense_9 (Dense)             (None, 2)                 386       
                                                                 
Total params: 169,560,386
Trainable params: 169,560,386
Non-trainable params: 0
_________________________________________________________________


In [21]:
model.compile(optimizer='adam', 
              loss=tf.losses.CategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [22]:
history = model.fit(
    mixture_stft, 
    epochs=10, 
    steps_per_epoch=500)

Epoch 1/10


ValueError: in user code:

    File "/home/rajesh/.local/lib/python3.10/site-packages/keras/engine/training.py", line 1051, in train_function  *
        return step_function(self, iterator)
    File "/home/rajesh/.local/lib/python3.10/site-packages/keras/engine/training.py", line 1040, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/home/rajesh/.local/lib/python3.10/site-packages/keras/engine/training.py", line 1030, in run_step  **
        outputs = model.train_step(data)
    File "/home/rajesh/.local/lib/python3.10/site-packages/keras/engine/training.py", line 889, in train_step
        y_pred = self(x, training=True)
    File "/home/rajesh/.local/lib/python3.10/site-packages/keras/utils/traceback_utils.py", line 67, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "/home/rajesh/.local/lib/python3.10/site-packages/keras/engine/input_spec.py", line 264, in assert_input_compatibility
        raise Va